In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from oneDtoyTask.oneDtoyTask import ToyTask, OneDfield, TwoWayComChannel
from agents.agent_Aria import Aria



In [2]:
epochs = 100
opt_params = {"lr":0.0005, "batch_size":10, "gamma":0.999}
agent0 = Aria(opt_params=opt_params).float()
agent1 = Aria(opt_params=opt_params).float()
np.random.seed(1)
field = OneDfield(speed=10)
Task = ToyTask(field=field, observationMappingFct=lambda x: (x>0.5).astype(int), comChannel=TwoWayComChannel())



In [5]:
obs, downlink_msgs = Task.reset()
losses = []
torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    a0, m0u =  agent0.select_action(obs[0], downlink_msgs[0])
    a1, m1u = agent1.select_action(obs[1], downlink_msgs[1])
    m0u_ = np.zeros(4)
    m1u_ = np.zeros(4)
    m0u_[m0u.item()] = 1
    m1u_[m1u.item()] = 1
    (obs_, downlink_msgs), r, done = Task.step([a0, a1], [m0u_, m1u_])
    loss0 = agent0.train_on_batch(obs[0], r[0])
    loss1 = agent1.train_on_batch(obs[1], r[1])
    obs = obs_
    losses.append([loss0, loss1])

returns_tensor tensor([ 1.4390,  0.7985,  0.8017,  0.1607,  0.1632, -0.4785, -1.1208, -1.7638])
self.saved_act_Logp [tensor([-0.5343], grad_fn=<SqueezeBackward1>), tensor([-0.5391], grad_fn=<SqueezeBackward1>), tensor([-0.5494], grad_fn=<SqueezeBackward1>), tensor([-0.5414], grad_fn=<SqueezeBackward1>), tensor([-0.8793], grad_fn=<SqueezeBackward1>), tensor([-0.5273], grad_fn=<SqueezeBackward1>), tensor([-0.8760], grad_fn=<SqueezeBackward1>), tensor([-0.5288], grad_fn=<SqueezeBackward1>), tensor([-0.5421], grad_fn=<SqueezeBackward1>), tensor([-0.8686], grad_fn=<SqueezeBackward1>)]
self.saved_msg_Logp [tensor([-1.3781], grad_fn=<SqueezeBackward1>), tensor([-1.3708], grad_fn=<SqueezeBackward1>), tensor([-1.3568], grad_fn=<SqueezeBackward1>), tensor([-1.2334], grad_fn=<SqueezeBackward1>), tensor([-1.3760], grad_fn=<SqueezeBackward1>), tensor([-1.5365], grad_fn=<SqueezeBackward1>), tensor([-1.3718], grad_fn=<SqueezeBackward1>), tensor([-1.3941], grad_fn=<SqueezeBackward1>), tensor([-1.3982]

RuntimeError: The size of tensor a (8) must match the size of tensor b (10) at non-singleton dimension 0